In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split  
from sklearn import preprocessing
import seaborn as sns 
import optuna 
import lightgbm as lgb
from sklearn.metrics import accuracy_score, roc_auc_score,precision_score, confusion_matrix,recall_score
from sklearn.model_selection import cross_val_score, train_test_split
import os 

/dshome/WoongLab/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_train_data.csv')


In [3]:
test_data=pd.read_csv('/dshome/WoongLab/heo/construction_oil/preprocessed_data/heart_test_data.csv')

In [4]:
train_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,27.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,2,2.0,1.0,0.0,0.0,13.0,4,1,0.0
1,1.0,1.0,1.0,25.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,0.0,10.0,5,6,0.0
2,0.0,0.0,1.0,24.0,1.0,0.0,0,1.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,7.0,3,6,0.0
3,1.0,1.0,1.0,24.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,2,0.0,30.0,1.0,0.0,11.0,3,3,0.0
4,0.0,0.0,1.0,20.0,0.0,0.0,0,1.0,1.0,1.0,...,0.0,1,15.0,0.0,0.0,0.0,2.0,5,5,0.0


In [5]:
test_data.head(5)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,Veggies,...,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income,HeartDiseaseorAttack
0,1.0,1.0,1.0,30.0,1.0,1.0,0,0.0,0.0,1.0,...,0.0,1,0.0,0.0,0.0,1.0,11.0,5,7,0.0
1,1.0,1.0,1.0,27.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,1,4.0,0.0,0.0,0.0,11.0,5,5,0.0
2,1.0,0.0,1.0,31.0,0.0,0.0,0,0.0,0.0,1.0,...,0.0,2,3.0,3.0,0.0,0.0,6.0,5,6,0.0
3,0.0,1.0,1.0,26.0,1.0,0.0,0,1.0,0.0,0.0,...,0.0,2,0.0,0.0,0.0,0.0,12.0,5,5,0.0
4,1.0,1.0,1.0,25.0,1.0,0.0,0,0.0,1.0,1.0,...,0.0,0,0.0,0.0,0.0,1.0,9.0,4,6,0.0


In [8]:
train_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker','GenHlth']

test_stage_features=['CholCheck','HvyAlcoholConsump','AnyHealthcare','Veggies','Fruits',
                     'NoDocbcCost','PhysActivity','Education','MentHlth','Smoker']

In [9]:
train_X=train_data.loc[:,train_stage_features] 
train_y=train_data['HeartDiseaseorAttack']
new_test_data_X=test_data[test_stage_features]
new_test_data_y=test_data['HeartDiseaseorAttack']

In [10]:
ratio=train_y.value_counts()[0]/train_y.value_counts()[1]

In [11]:
# train에는 있는데 test data에는 없는 컬럼 찾기

train_data_columns=np.array(train_X.columns)
test_data_columns=np.array(new_test_data_X.columns)
np.setdiff1d(train_data_columns,test_data_columns)

array(['GenHlth'], dtype=object)

In [12]:
# 변수 AL을 예측하는 모형 만들어 주기
import optuna
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

test_data_columns=test_stage_features
y='GenHlth'

train_X_new=train_X.loc[:,test_data_columns]
train_y_new=train_X.loc[:,y]



# Objective 함수 정의
def objective(trial):
    
    # 하이퍼파라미터 탐색할 공간 정의
    params = {
        'n_estimators': trial.suggest_int('n_estimators',100,1000),
        'max_depth': trial.suggest_int('max_depth',3,9),
        'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
    }
    
    # LGBMRegressor 모델 객체 생성
    model = LGBMRegressor(**params, random_state=42)
    
    # 교차검증 수행하여 모델 성능 측정
    scores = -1 * cross_val_score(model, train_X_new, train_y_new,
                                  cv=5, scoring='neg_mean_squared_error')
    
    # 교차검증 평균 점수 리턴
    return np.mean(scores)


# Optuna study 생성
study = optuna.create_study(direction='minimize')

# study 실행 (n_trials는 시도 횟수)
study.optimize(objective, n_trials=10)

# 최적화된 하이퍼파라미터 값 출력
print(study.best_params)



[I 2023-04-25 21:28:08,235] A new study created in memory with name: no-name-9958434d-9c3a-4f15-9c55-f1fe5b5b5223
/tmp/ipykernel_612690/3006524672.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 21:28:18,448] Trial 0 finished with value: 0.8915086589539756 and parameters: {'n_estimators': 571, 'max_depth': 9, 'learning_rate': 0.014566582806380549}. Best is trial 0 with value: 0.8915086589539756.
/tmp/ipykernel_612690/3006524672.py:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate': trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04

{'n_estimators': 745, 'max_depth': 3, 'learning_rate': 0.02298077264792893}


In [13]:
# Print best hyperparameters and auc

print(f'Best hyperparameters: {study.best_params}')
print(f'Best RMSE: {study.best_value:.4f}')

Best hyperparameters: {'n_estimators': 745, 'max_depth': 3, 'learning_rate': 0.02298077264792893}
Best RMSE: 0.8908


In [14]:
(est,depth,rate)=study.best_params.values()

In [15]:
def bestreg_parametertuning(rate,depth,est,test_data_columns,y):
    best_lgbmreg=LGBMRegressor(learning_rate=rate,max_depth=depth,n_estimators=est,random_state=42)
    best_lgbmreg.fit(train_X[test_data_columns], train_X[y])
    new_test_data_X[y]=best_lgbmreg.predict(new_test_data_X[test_data_columns])

In [16]:
bestreg_parametertuning(rate,depth,est,test_data_columns,'GenHlth')

/tmp/ipykernel_612690/3339438919.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_test_data_X[y]=best_lgbmreg.predict(new_test_data_X[test_data_columns])


In [17]:
new_test_data_X

,CholCheck,HvyAlcoholConsump,AnyHealthcare,Veggies,Fruits,NoDocbcCost,PhysActivity,Education,MentHlth,Smoker,GenHlth
0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,5,0.0,1.0,1.697328
1,1.0,0.0,1.0,1.0,0.0,0.0,0.0,5,4.0,0.0,1.731286
2,1.0,0.0,1.0,1.0,0.0,0.0,0.0,5,3.0,0.0,1.674033
3,1.0,0.0,1.0,0.0,0.0,0.0,1.0,5,0.0,1.0,1.387262
4,1.0,0.0,1.0,1.0,1.0,0.0,0.0,4,0.0,1.0,1.864076
...,...,...,...,...,...,...,...,...,...,...,...
50731,1.0,0.0,1.0,1.0,1.0,0.0,0.0,5,1.0,0.0,1.507892
50732,1.0,0.0,0.0,1.0,1.0,1.0,0.0,3,10.0,0.0,2.580546
50733,1.0,0.0,1.0,0.0,0.0,0.0,0.0,3,10.0,0.0,2.362353
50734,1.0,0.0,1.0,1.0,1.0,0.0,0.0,5,0.0,0.0,1.466617


In [18]:
import lightgbm as lgb

In [19]:
def objective(trial):
    # Define hyperparameters to optimize 
    params={
        'boosting_type':'gbdt',
        'objective':'binary',
        'metric':'binary_logloss',
        'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
        'max_depth':trial.suggest_int('max_depth',3,9),
        'n_estimators':trial.suggest_int("n_estimators",100,1000)
    }
    # Train and evaluate model 
    lgb_cv=lgb.LGBMClassifier(**params, random_state=42,scale_pos_weight=ratio)
    scores=cross_val_score(lgb_cv,train_X,train_y,cv=5,scoring='roc_auc')
    auc=scores.mean()
    return auc 

# Define study object and optimize 

study=optuna.create_study(direction='maximize',study_name='lgb_boost_opt',load_if_exists=True)
study.optimize(objective, n_trials=50)

# Print best hyperparameters and auc
print(f'Best hyperparameters: {study.best_params}')
print(f'Best AUC: {study.best_value:.4f}')

[I 2023-04-25 21:32:53,103] A new study created in memory with name: lgb_boost_opt
/tmp/ipykernel_612690/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 21:33:02,431] Trial 0 finished with value: 0.7509261593118726 and parameters: {'learning_rate': 0.06902178520317526, 'max_depth': 6, 'n_estimators': 574}. Best is trial 0 with value: 0.7509261593118726.
/tmp/ipykernel_612690/2141867322.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'learning_rate':trial.suggest_loguniform('learning_rate',0.01,0.1),
[I 2023-04-25 21:33:11,892] Trial 1 finished w

Best hyperparameters: {'learning_rate': 0.028023839166238077, 'max_depth': 3, 'n_estimators': 627}
Best AUC: 0.7578


In [20]:
lgb=lgb.LGBMClassifier(learning_rate= 0.028023839166238077,max_depth=3,n_estimators=627,scale_pos_weight=ratio,random_state=42)

In [21]:
lgb.fit(train_X,train_y)

LGBMClassifier(learning_rate=0.028023839166238077, max_depth=3,
               n_estimators=627, random_state=42,
               scale_pos_weight=9.617557811028565)

In [22]:
pred=lgb.predict_proba(new_test_data_X)[:,1]

In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

def get_clf_prob(y_test, probability):
  pred=np.where(probability > 0.50,1,0)
  confusion=confusion_matrix(y_test, pred)
  accuracy=accuracy_score(y_test,pred)
  precision=precision_score(y_test,pred) 
  recall=recall_score(y_test,pred) 
  # F1 스코어 추가 
  f1=f1_score(y_test,pred,average='macro')
  Roc_score=roc_auc_score(y_test,probability)
  print('임계값: ', 0.5) 
  print('오차행렬')
  print(confusion) 
  # f1 score print 추가 
  print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f}, F1:{3:.4f}, AUC:{4: .4f}'.format(accuracy,precision,recall,f1,Roc_score))

In [24]:
get_clf_prob(new_test_data_y,pred) 

임계값:  0.5
오차행렬
[[33170 12787]
 [ 2409  2370]]
정확도: 0.7005, 정밀도: 0.1564, 재현율: 0.4959, F1:0.5257, AUC: 0.6585
